In [20]:
# 1. Install libomp: brew install libomp
# 2. export environment variables for xgboost: export LIBRARY_PATH="/opt/homebrew/opt/libomp/lib:$LIBRARY_PATH"
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf

# PyPortfolioOpt
from pypfopt import risk_models, expected_returns
from pypfopt import black_litterman
from pypfopt import BlackLittermanModel, plotting
from pypfopt import EfficientFrontier, objective_functions
from pypfopt.discrete_allocation import DiscreteAllocation

# XGBoost
import xgboost as xgb

# Exempelvis för att tysta varningar:
import warnings
warnings.filterwarnings("ignore")

# FGI Data gathering

In [21]:
# Normal FGI
df_fgi = pd.read_csv(
    "datasets/FGIfromGithub2011-2020.csv",
    usecols=["Date","Fear Greed"],  # vi vill bara ha datum och FGI
    parse_dates=["Date"]
)
df_fgi.rename(columns={"Fear Greed": "FGI"}, inplace=True)
df_fgi.set_index("Date", inplace=True)  # sätt datumet som index
df_fgi.sort_index(inplace=True)         # sortera på datum
print(df_fgi.head())

            FGI
Date           
2011-01-03   68
2011-01-04   68
2011-01-05   67
2011-01-06   64
2011-01-07   63


In [22]:
# Crypto FGI
df_fgi_crypto = pd.read_csv(
    "datasets/CryptoFEI.csv",
    parse_dates=["Date"],
    dayfirst=True  # talar om att '17-02-2025' är 17:e februari
)
df_fgi_crypto.rename(columns={"Fng_value": "FGI_crypto"}, inplace=True)
df_fgi_crypto.set_index("Date", inplace=True)
df_fgi_crypto.sort_index(inplace=True)
print(df_fgi_crypto.head())

            FGI_crypto Fng_classification
Date                                     
2018-02-01          30               Fear
2018-02-02          15       Extreme Fear
2018-02-03          40               Fear
2018-02-04          24       Extreme Fear
2018-02-05          11       Extreme Fear


In [ ]:
# Gold and silver ratio
df_goldsilver = pd.read_csv(
    "datasets/Gold-Silver-Ratio-since-1693.csv",
    sep=";", 
    decimal=",",         # om kommatecken är decimalavskiljare
    parse_dates=["DateTime"]
)

# Byt gärna kolumnnamn
df_goldsilver.rename(columns={"Gold/Silver": "gold_silver_ratio"}, inplace=True)

# Sätt index till datum
df_goldsilver.set_index("DateTime", inplace=True)
df_goldsilver.sort_index(inplace=True)
df_goldsilver = df_goldsilver[df_goldsilver.index >= "2010"]
print(df_goldsilver.head())

                     gold_silver_ratio
DateTime                              
1693-12-31 00:00:28              14.83
1694-12-31 00:00:28              14.87
1695-12-31 00:00:28              15.02
1696-12-31 00:00:28              15.00
1697-12-31 00:00:28              15.20


# Data gathering for ETFs

In [24]:
ETFs = ["SPY", "BND", "GLD", "HODL"] # Maybe change HODL
ohlc = yf.download(ETFs, period="15y")
closing_prices = ohlc["Close"].dropna(how="all")
print(closing_prices.tail())

[*********************100%***********************]  4 of 4 completed


Ticker            BND         GLD       HODL         SPY
Date                                                    
2025-04-04  73.820000  279.720001  23.750000  505.279999
2025-04-07  72.930000  273.709991  22.080000  504.380005
2025-04-08  72.510002  275.200012  21.709999  496.480011
2025-04-09  72.680000  285.380005  23.250000  548.619995
2025-04-10  72.099998  292.350006  22.500000  524.580017
